In [2]:
# MESMA script - maybe a clumsy attempt as I am just learning about it and how to use it. 
# assumes modules have already been added to environment 

import numpy as np
import matplotlib.pyplot as plt
import MESMA
from sentinelapi import SentinelAPI, read_geojson, geojson_to_wkt

# assumes rasterio, matplotlib, sentinelapi (sentinelsat), etc. are installed

# Get input
scene_id = input("Enter Sentinel-2 scene ID: ")

# Create a MESMA object
mesma = MESMA()

# Download scene
api = SentinelAPI("<username>", "<password>", "https://scihub.copernicus.eu/dhus")
footprint = geojson_to_wkt(read_geojson("<geojson_file_path>"))
products = api.query(footprint, date=("2020-01-01", "2020-01-31"), platformname='Sentinel-2', processinglevel='Level-2A')
product_id = list(products.keys())[0]
api.download(product_id)

# Load image data
product_path = api.to_geotiff(product_id)
data = np.load(product_path)

# Spectral unmixing
unmixed_data = mesma.unmix(data)

# Save unmixed data
np.save("unmixed_data.npy", unmixed_data)

# Plot  
plt.imshow(unmixed_data)
plt.show()

ModuleNotFoundError: No module named 'MESMA'

In [ ]:
# more endmember_extraction 

import rasterio
import numpy as np
from spectral import *
from spectral import endmember_extraction 

# Load Sentinel-2 data
with rasterio.open("sentinel2.tif") as src:
    data = src.read()
    profile = src.profile

# Extract bands 
bands = data[[2, 3, 4, 8], :, :]

# endmember extraction
endmembers = endmember_extraction(bands.reshape(bands.shape[0], -1), algorithm='NFINDR', no_of_endmembers=10)

# Reshape endmembers back to original shape
endmembers = endmembers.reshape(-1, bands.shape[1], bands.shape[2])

# Save output as a geotiff file using rasterio
with rasterio.open("mesma_output.tif", "w", **profile) as dst:
    dst.write(endmembers.astype(np.float32), 1)

In [ ]:
# using existing spectral libraries with MESMA:

import numpy as np
from mesma import MESMA
from spectral_libraries import Landsat8 

# or any of  available spectral libraries/modules 
# Rasterio for geospatial raster data from Landsat 8, Sentinel-2, etc. 
# Spectral for spectral analysis of Landsat 8, Sentinel-2, MODIS, ASTER, WorldView-2, RapidEye, VNIR-Compact, Hyperion, HyMap, and EO-1 Hyperion.
# GDAL for raster and vector formats
# SentinelAPI client for Copernicus Open Access Hub to search for and download Sentinel-2 data,
# PyHDF Python interface for HDF4/HDF5 for sensors that produce HDF4 or HDF5 files.

 
# for Sentinel-2:
from spectral_libraries import Sentinel2

# Load  image data
data = np.load("image.npy")

# Load  spectral library
spectral_library = Sentinel2()

# Identify  vegetation endmembers
endmembers = MESMA.unmix(data, spectral_library)

# Save 
np.save("endmembers.npy", endmembers)

# Use cross-validation set to evaluate  performance of unmixing model
accuracy = MESMA.cross_validate(data, endmembers)

print(" accuracy of  unmixing model is {}%".format(accuracy))


In [ ]:
# possible alternate - I am learning about the spectral module - have not used it before but it contains a lot 
# including spectral libraries for several satellites, ECOSTRESS, algorithms for supervised and unsupervised learning
# as well as dimensionality reduction and different algorithms for endmember extraction. Endmember extraction algorithms 
# (spectral.endmember module): Vertex Component Analysis (VCA), Pixel Purity Index (PPI), N-FINDR (N-dimensional Feature Space for 
# Discrimination of Materials), Automatic Target Generation Process (ATGP), Minimum Noise Fraction (MNF) - which can be used to identify 
# endmembers present in a spectral image, which can be used for spectral unmixing.

import spectral

# Load data
data = spectral.open_image('data.hdr')

# Extract endmembers using VCA algorithm
endmembers = spectral.endmember.vca(data, n_endmembers=5)

# Display endmembers
spectral.imshow(endmembers)

In [ ]:
# Load a scene with Sentinel API 

import numpy as np
import matplotlib.pyplot as plt

from sentinelsat import SentinelAPI
from mesma import MESMA

# Get scene id
scene_id = input("scene ID: ")

# Create a SentinelAPI object 
api = SentinelAPI("username", "password")

# Download scene
products = api.get_products(scene_id)
product = products[0]
product.download()

# Load image
data = np.load(product.path)

# Create a MESMA object
n_endmembers = 10  # specify the number of endmembers to extract
mesma = MESMA(n_endmembers)

# Spectral unmixing
unmixed_data = mesma.unmix(data)

# Save
mesma.save_unmixed_data(unmixed_data, output_filename)

# Plot abundance maps for each endmember
for i in range(unmixed_data.shape[2]):
    plt.imshow(unmixed_data[:, :, i], cmap='gray', extent=[0, data.shape[1], data.shape[0], 0])
    plt.title("Abundance map for endmember " + str(i+1))
    plt.show()



In [ ]:
# I was working on some ideas for using PCA, bootstrapping or test train split to speed things up but I'm not ready to share those yet

# Use PCA - I don't know if we want to do this or not, but just in case. Spectral has built-in dimensionality reducation
# but I'm still learning about it

import numpy as np
from sklearn.decomposition import PCA
import rasterio

# Load raster data
data = np.load("raster_data.npy")

# Reshape to two-dimensional array
n_bands, n_rows, n_cols = data.shape
data_2d = data.reshape(n_bands, n_rows * n_cols).T

# Perform PCA 
pca = PCA(n_components=3)
pca.fit(data_2d)

# Calculate and print explained variance ratio for principal components
explained_variance_ratio = pca.explained_variance_ratio_

for i, ratio in enumerate(explained_variance_ratio):
    print("Principal component %d explains %.2f%% of  variance" % (i + 1, ratio * 100))

# Transform data using PCA transformation matrix
data_transformed_2d = pca.transform(data_2d)

# Reshape transformed data to original shape
data_transformed = data_transformed_2d.T.reshape(3, n_rows, n_cols)

# Save transformed data as a geotiff file
with rasterio.open("pca_transformed_data.tif", "w", **profile) as dst:
    dst.write(data_transformed.astype(np.float32)) 